In [0]:
%sh apt-get update && apt-get install -y coinor-cbc

In [0]:
%pip install -r ./requirements.txt --quiet
dbutils.library.restartPython()

In [0]:
import os
import mlflow
import random
import numpy as np
import pandas as pd
import cloudpickle
import pyomo.environ as pyo
import scripts.utils as utils

Build the three-tier network exactly with “close” Tier 3 → Tier 2 links (each Tier-3 node picks the nearest Tier-2 node, with a random tie-break when it sits between two). Guarantee no isolation – every Tier-1 node receives ≥ 1 edge from Tier-2, and every Tier-2 node receives ≥ 1 edge from Tier-3 (the assertion would fail otherwise).

In [0]:
# Generate a synthetic 3-tier network dataset for optimization 
dataset = utils.generate_data(N1=5, N2=10, N3=20)
#dataset = utils.generate_data(N1=200, N2=500, N3=1000)

Visualise the tiers cleanly with distinctive shapes (●, ■, ▲) and a layered layout.

In [0]:
# Visualizes the 3-tier network
utils.visualize_network(dataset)

In [0]:
print("Tier sizes  :", len(dataset['tier1']), len(dataset['tier2']), len(dataset['tier3']))
print("Edges       :", len(dataset['edges']))
print("f:", {j: dataset['f'][j] for j in dataset['tier1']})
print("d:", {j: dataset['d'][j] for j in dataset['tier1']})
print("s:", {j: dataset['s'][j] for j in dataset['tier1']})
print("c:", {j: dataset['c'][j] for j in dataset['tier2']})

In [0]:
# Assign a random ttr to each disrupted node
random.seed(777)
disrupted_nodes = {node: random.randint(1, 10) for node in dataset['tier2'] + dataset['tier3']}

objectives = []
for disrupted_node in disrupted_nodes:
    disrupted = [disrupted_node]
    df = utils.build_and_solve_multi_tier_ttr(dataset, disrupted, disrupted_nodes[disrupted_node])
    objectives.append(df)

In [0]:
objectives = pd.concat(objectives, ignore_index=True)
display(objectives)

Databricks visualization. Run in Databricks to view.

In [0]:
highest_risk_nodes = objectives.sort_values(by="objective_value", ascending=False)[0:5]
highest_risk_nodes

In [0]:
model_pickled = highest_risk_nodes["pickled_model"].values[0]  
model = cloudpickle.loads(model_pickled)

records = []
for v in model.component_data_objects(ctype=pyo.Var, active=True):
    idx  = v.index()
    record  = {
        "var_name"  : v.parent_component().name,
        "index"     : idx,
        "value"     : pyo.value(v),
        "lb"        : v.lb,
        "ub"        : v.ub,
        "fixed"     : v.fixed,
    }
    records.append(record)

pd.DataFrame.from_records(records)